# Vehicle Example of using classes to build types in Python

Using the example of different types of vehicles, we can explore how classes add *Constraint, Capability and Meaning* to our data.

First lets create a Vehicle type with the *Capability* to have a name, a colour and cause crashes. 
We will also give this Vehicle the *Capability* to be added to other vehicles. 
At this point in time when a vehicle is added to another vehicle this will cause a crash

We will also add a `__str__` method to return a string representation of this object

In [ ]:
class Vehicle(object):
    def __init__(self, name, colour):
        self.name = name
        self.colour = colour

    class Crash(Exception):
        pass

    def __add__(self, value):
        if isinstance(value, Vehicle):
            raise self.Crash(f"You ({self.name}) crashed into another vehicle ({value.name})")
        raise NotImplementedError

    def __str__(self):
        return f"{self.colour} {self.name}"


## Extending our class with inheritance

Now say we want to create a specific type of vehicle, a car. 
We can use our existing base to create a subclass which has all the *Capabilities and Constraints* of its parent, with extra options. 
This is called inheritance. 

in this case we will give the car the *Capability* of having a number of doors.


In [ ]:
class Car(Vehicle):
    def __init__(self, name, colour, doors):
        """ cars have doors """
        self.doors = int(doors)
        super().__init__(name, colour)


In [ ]:
car_1 = Car("Hyundai i30", "blue", 5)
car_2 = Car("Fiat 500", "grey", 3)

Now lets try and add these two cars together

In [ ]:
car_2 + car_1

Let's check the type of our instance (car_1)

In [ ]:
type(car_1)

## Further extension

We can now create other types of vehicles. 
Let's create a Motorcycle class. We can't extend car, because the motorcycle isn't a type of car, it has no doors. 
It is however a type of vehicle, so let's extend our vehicle type again. 

In [ ]:
class Motorcycle(Vehicle):
    # bikes don't have doors
    pass

In [ ]:
bike_1 = Motorcycle("Kawasaki Ninja", "Green")

In [ ]:
car_1 + bike_1

## Overriding methods

So it turns out that we can't add our Motorcycle to our car. 
However what if we could? 
Maybe if we had some other form of vehicle, in this case lets make a trailer. 

A trailer is a vehicle which will have the ability to contain certain things. 
We are going to give this vehicle the *Capability* to have things added to it, however we are also going to give it the *Constraint* of "can_carry" which is a list of classes which will be able to fit onto this trailer. 
To do this we are going to *Override* our __add__ method from the parent class, and we will add some code to deal with addition of items we can carry. 
This *Constraint* will help us keep our data clean, if we do not have the *Capability* to carry this type of object, we will use the `super()` command to revert back to the behaviour in the parent class.

In [ ]:
class Trailer(Vehicle):

    can_carry = (Motorcycle,)

    def __init__(self, name, colour):
        self.contains = None
        super().__init__(name, colour)

    def __add__(self, value):
        if type(value) in self.can_carry:
            self.contains = value
            print(f"loaded {value.name}")
            return True
        super().__add__(value)

    def __str__(self):
        if self.contains:
            return f"{self.colour} Trailer {self.name} containing {str(self.contains)}"
        return f"{self.colour} Trailer {self.name}"


In [ ]:
bike_1 + 3

In [ ]:
tailer_1 = Trailer('rusty', 'rust')

In [ ]:
tailer_1 + car_1

In [ ]:
trailer_1 = tailer_1

In [ ]:
trailer_1 + bike_1

In [ ]:
trailer_1.contains

In [ ]:
id(bike_1)

In [ ]:
trailer_1.contains.name

## Grandchildren

We don't always need to go back to the base to extend and inherit from, we can do it from anywhere. 
Lets say we want to create a truck. A truck is a type of car, it has the same *Constraints* as a car, but with some extra *Capabilities* such as the ability to tow. 
Let's create a Truck by extending the Car class. 
We will give it the ability to tow. 
We will record what it is towing by creating an attribute called `towing` to store the object it tows in.
We will use `super()` again to use all of the inherited init commands from the Car parent class. 
We will *Override* the parent class `__add__` method to allow for hooking up an oject it `can_tow`, however we will pass any unmatched object back to the parent class to handle. 

We will also update the `__str__` method to print not just the colour and name, but also what is being towed, if something is being towed. 

In [ ]:
class Truck(Car):

    can_tow = (Trailer,)

    def __init__(self, name, colour, doors):
        """ cars have doors """
        self.towing = None
        super().__init__(name, colour, doors)

    def __add__(self, value):
        if type(value) in self.can_tow:
            if not self.towing:
                self.towing = value
                print(f"hooked up {value.name}")
            else:
                print(f"already towing {self.towing.name}")
            return None
        super().__add__(value)

    def __str__(self):
        return f"{self.colour} Truck {self.name} towing {str(self.towing)}"



In [ ]:
truck_1 = Truck('Dodge Ram', 'black', 4)

In [ ]:
truck_1

In [ ]:
str(truck_1)

In [ ]:
truck_1 + trailer_1

In [ ]:
truck_1.towing

In [ ]:
str(truck_1.towing)

In [ ]:
str(truck_1.towing.contains)

In [ ]:
truck_1.towing.name

In [ ]:
truck_1.towing.contains.name

In [ ]:
dir(truck_1)

In [ ]:
type(truck_1).__bases__

In [ ]:
type(truck_1).__mro__

## Not ever vehicle goes on the road

Lets mix things up by creating a different type of vehicle. 
Let's create a ship.

We are going to give our ship the *Capability* to contain multiple objects, for this we will add a `contains` attribute which will be a list.
We will also give our Ship the *Capability* of being `afloat` which we will default to `True`
We will also give our Ship the *Constraint* of having a capacity, as different ships will have different capacities. 
To allow us to remove items from the ship to not meat this *Constraint* we will need the *Capability* of being able to subtract objects from the vehicle, to do this we will *Override* the `__sub__` method. 

For fun we will also give our ship a `Sink` exception. 
If we put too many objects on this ship, the ship will set `afloat` to `False` and then raise the `Sink` exception.

We can also give the ship the ability to be floated, for fun we will *Override* the `float()` function in python by using the `__float__` method. 
Normally this is reserved for numbers, but in python it doesn't matter if something is a number or not, what matters is what *Capability* and *Constraint* it has, as python cares about **WHAT the data can do** more than it cares about **What the data actually is** this is the magic of *Duck Typing* 

In [ ]:
class Ship(Vehicle):
    capacity = 1
    def __init__(self, name, colour):
        self.contains = []
        self.afloat = True
        super().__init__(name,colour)

    class Sink(Exception):
        pass

    def __add__(self, value):
        if not self.afloat:
            raise ValueError("can not load item to sunken ship")
        if not isinstance(value, Vehicle):
            raise ValueError("Not a vehicle")
        if len(self.contains) < self.capacity:
            self.contains.append(value)
            print(f"loaded {value.name} onto {self.name}")
        else:
            self.afloat = False
            raise self.Sink(f"you sunk my {str(self)}")
            
    def __sub__(self, idx):
        """ remove a thing from the ship"""
        x = self.contains.pop(idx)
        print(f"removed {x.name} from {self.name}")
        
    def __float__(self):
        self.afloat = True
        print(f"floated {self.name}")
        return float(0)
        
    @property
    def inventory(self):
        for idx, thing in enumerate(self.contains):
            print(f"slot {idx:<2} {str(thing)}")
        

### A ship that can carry one thing is boring

So lets create a bigger ship. 
Lets create a *Type* of vehicle called a Ferry, which is able to carry many more vehicles.

In [ ]:
class Ferry(Ship):
    # naval architect says it can carry 10
    capacity = 10


In [ ]:
boaty_mc_boatface = Ferry("Boaty McBoatface", "Rainbow")

In [ ]:
str(boaty_mc_boatface)

In [ ]:
boaty_mc_boatface + truck_1

In [ ]:
boaty_mc_boatface.contains

In [ ]:
boaty_mc_boatface.inventory

In [ ]:
boaty_mc_boatface + 5

In [ ]:
boaty_mc_boatface.contains[0].towing.contains.name

In [ ]:
# show the Method Resolution Order
type(boaty_mc_boatface).__mro__

In [ ]:
str(boaty_mc_boatface)

In [ ]:
boaty_mc_boatface.__str__

In [ ]:
boaty_mc_boatface.__str__()

In [ ]:
#  attempt to return a list of valid attributes for that object
dir(boaty_mc_boatface)

In [ ]:
for x in range(10):
    boaty_mc_boatface + Car('Generic Car', "vanilla", 4)

In [ ]:
boaty_mc_boatface.inventory

In [ ]:
boaty_mc_boatface + car_1

In [ ]:
boaty_mc_boatface.afloat

In [ ]:
boaty_mc_boatface - 9

In [ ]:
boaty_mc_boatface.afloat

In [ ]:
boaty_mc_boatface == 1

In [ ]:
float(boaty_mc_boatface)

In [ ]:
str(boaty_mc_boatface)

In [ ]:
boaty_mc_boatface.contains